# T1055.011 - Extra Window Memory Injection
Adversaries may inject malicious code into process via Extra Window Memory (EWM) in order to evade process-based defenses as well as possibly elevate privileges. EWM injection is a method of executing arbitrary code in the address space of a separate live process. 

Before creating a window, graphical Windows-based processes must prescribe to or register a windows class, which stipulate appearance and behavior (via windows procedures, which are functions that handle input/output of data).(Citation: Microsoft Window Classes) Registration of new windows classes can include a request for up to 40 bytes of EWM to be appended to the allocated memory of each instance of that class. This EWM is intended to store data specific to that window and has specific application programming interface (API) functions to set and get its value. (Citation: Microsoft GetWindowLong function) (Citation: Microsoft SetWindowLong function)

Although small, the EWM is large enough to store a 32-bit pointer and is often used to point to a windows procedure. Malware may possibly utilize this memory location in part of an attack chain that includes writing code to shared sections of the process’s memory, placing a pointer to the code in EWM, then invoking execution by returning execution control to the address in the process’s EWM.

Execution granted through EWM injection may allow access to both the target process's memory and possibly elevated privileges. Writing payloads to shared sections also avoids the use of highly monitored API calls such as <code>WriteProcessMemory</code> and <code>CreateRemoteThread</code>.(Citation: Elastic Process Injection July 2017) More sophisticated malware samples may also potentially bypass protection mechanisms such as data execution prevention (DEP) by triggering a combination of windows procedures and other system functions that will rewrite the malicious payload inside an executable portion of the target process.  (Citation: MalwareTech Power Loader Aug 2013) (Citation: WeLiveSecurity Gapz and Redyms Mar 2013)

Running code in the context of another process may allow access to the process's memory, system/network resources, and possibly elevated privileges. Execution via EWM injection may also evade detection from security products since the execution is masked under a legitimate process. 

## Atomic Tests

### Atomic Test #1 - Process Injection via Extra Window Memory (EWM) x64 executable
Hooks functions of main process to inject a payload via Extra Window Memory (EWM) injection technique
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: T1055.011x64.exe and payload must exist on disk at specified location (#{exe_binary} and #{payload_file})
##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1055.011\bin\T1055.011_#{arch}.exe) {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path PathToAtomicsFolder\T1055.011\bin\T1055.011_#{arch}.exe) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1055.011/bin/T1055.011_x64.exe" -OutFile "PathToAtomicsFolder\T1055.011\bin\T1055.011_#{arch}.exe" -UseBasicParsing
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1055.011/bin/payload.exe_x64.bin" -OutFile "PathToAtomicsFolder\T1055.011\bin\payload.exe_#{arch}.bin" -UseBasicParsing
```

In [ ]:
Invoke-AtomicTest T1055.011 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
PathToAtomicsFolder\T1055.011\bin\T1055.011_#{arch}.exe```

In [ ]:
Invoke-AtomicTest T1055.011 -TestNumbers 1

#### Cleanup: 
```powershell
Get-Process -Name Notepad -ErrorAction SilentlyContinue | Stop-Process -Force```

In [ ]:
Invoke-AtomicTest T1055.011 -TestNumbers 1 -Cleanup

## Detection
Monitor for API calls related to enumerating and manipulating EWM such as GetWindowLong (Citation: Microsoft GetWindowLong function) and SetWindowLong (Citation: Microsoft SetWindowLong function). Malware associated with this technique have also used SendNotifyMessage (Citation: Microsoft SendNotifyMessage function) to trigger the associated window procedure and eventual malicious injection. (Citation: Elastic Process Injection July 2017)